In [1]:
import itertools
import gc
from bib import *
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras import backend as K 
from keras import  activations
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Dense, Activation, regularizers,LSTM,Bidirectional,CuDNNLSTM
import tensorflow as tf
from  tqdm import tqdm_notebook
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [2]:
xtrains,ytrains,xvals,yvals,xtest,ytest=rnnSplit(15, ('CHL 1','THERM 1','SSH','SR','WS'),
                                               ['CHL '+ str(i) for i in range(2,19)])
ytrains,yvals,ytest=applyToy(lambda x:np.log(x)/np.log(10),ytrains,yvals,ytest)

In [21]:
def generate():
    layers=[]
    layers.append(Bidirectional(LSTM(100,input_shape=(15,5),activation='softplus',return_sequences=True),
                                merge_mode='ave',input_shape=(15,5)))
    layers.append(Bidirectional(LSTM(100,input_shape=(15,5),activation='softplus',return_sequences=True),
                                merge_mode='ave',input_shape=(15,5)))
    layers.append(Dense(17,activation='linear',kernel_regularizer=regularizers.l2(0.01)))
    model = Sequential(layers)
    model.compile(optimizer='rmsprop',
              loss='mse')
    model._make_predict_function()
    return model
callbacks = [EarlyStopping(monitor='val_loss', patience=150)]

In [8]:
def validate(xtrains,ytrains,xvals,yvals):
    fold_evaluation=[]
    for xt,yt,xv,yv in tqdm_notebook(zip(xtrains,ytrains,xvals,yvals),total=len(xtrains)):
        model=generate()
        
#         mean=xt.mean()
#         std=xt.std()
        
#         xt=(xt-mean)/std
#         xv=(xv-mean)/std
        
        
        yt=np.exp(yt*np.log(10))
        yv=np.exp(yv*np.log(10))        
        
        model.fit(xt,yt,
                  #epochs=10000000,
                  epochs=10,
                  callbacks=callbacks,
                  verbose=0,
                  batch_size=73*10, 
                  validation_data=(xv,yv))
        
        yp = model.predict(xv)
        #check error in original space
        
#         yp=np.exp(yp*np.log(10))
#         yv=np.exp(yv*np.log(10))       
        fold_evaluation.append(evaluate(yp.reshape(-1),yv.reshape(-1)))
        print(evaluate(yp.reshape(-1),yv.reshape(-1)))
        
    return np.mean(fold_evaluation),np.std(fold_evaluation )


In [6]:
K.clear_session()
layers=[]
layers.append(Bidirectional(LSTM(100,input_shape=(15,5),activation='softplus',return_sequences=True),
                            merge_mode='ave',input_shape=(15,5)))
layers.append(Bidirectional(LSTM(100,input_shape=(15,5),activation='softplus',return_sequences=True),
                            merge_mode='ave',input_shape=(15,5)))
layers.append(Dense(17,activation='linear',kernel_regularizer=regularizers.l2(0.01)))
model = Sequential(layers)
model.compile(optimizer='rmsprop',
          loss='mse')


model.fit(xtrains[0],np.exp(ytrains[0]*np.log(10)),
          epochs=10,
          callbacks=callbacks,
          verbose=1,
          batch_size=73*10, 
          validation_data=(xvals[0],np.exp(yvals[0]*np.log(10))))

Train on 531 samples, validate on 59 samples
Epoch 1/10
531/531 [==============================] - 4s 8ms/step - loss: 367.2632 - val_loss: 238.8194
Epoch 2/10
531/531 [==============================] - 0s 929us/step - loss: 234.3964 - val_loss: 182.2807
Epoch 3/10
531/531 [==============================] - 0s 926us/step - loss: 177.3677 - val_loss: 119.4188
Epoch 4/10
531/531 [==============================] - 0s 936us/step - loss: 116.4657 - val_loss: 50.0241
Epoch 5/10
531/531 [==============================] - 0s 917us/step - loss: 47.4946 - val_loss: 37.2660
Epoch 6/10
531/531 [==============================] - 0s 919us/step - loss: 35.0788 - val_loss: 26.9930
Epoch 7/10
531/531 [==============================] - 0s 925us/step - loss: 25.6705 - val_loss: 21.3962
Epoch 8/10
531/531 [==============================] - 1s 1ms/step - loss: 20.3335 - val_loss: 16.8512
Epoch 9/10
531/531 [==============================] - 0s 933us/step - loss: 15.4511 - val_loss: 14.9411
Epoch 10/10
531/

In [10]:

K.clear_session()

# print('ACT :'+ str(act))
error,std=validate(xtrains,ytrains,xvals,yvals)
print('\t error: %.5f +- %.5f' % (error,std))

1.724900106494487
0.9415060403967487
0.511121903978202
2.6795160535462657
1.0799391291770166
0.4610785461047105
0.7122998667203135
0.6074455653495955
0.40586797951703096
0.1831031903402495
0.36910353747986213
0.17429001280625195
0.18516877777210872
0.24273128011719594
0.14548139213348774
	 error: 0.69490 +- 0.67300


In [22]:
model = generate()

model.fit(xtrains[14], ytrains[14], epochs=10,
                  callbacks=callbacks,
                  verbose=0,
                  batch_size=73*10, 
                  validation_data=(xvals[14],yvals[14]))

In [32]:
yp = model.predict(xtest)
print(evaluate(np.exp(yp.reshape(-1)*np.log(10)),np.exp(ytest.reshape(-1)*np.log(10))))

0.14357596606025352
